# Capítulo 3: Regresión lineal

<h2> Contenido <span class="tocSkip"></span></h2>
<hr>
<div class="toc"><ul class="toc-item">
    <li><span>
        <a href="#1.-Introduction" data-toc-modified-id="1.-Introduction-1">1. Introduction</a>
    </span></li>
    <li><span>
        <a href="#2.- Modelo estadístico" data-toc-modified-id="2.-Modelo estadístico2">2. Modelo estadístico</a>
    </span></li>
    <li><span>
        <a href="#3.-Exercises" data-toc-modified-id="3.-Exercises-3">3. Exercises</a>
    </span></li>
</ul></div>

## Objetivos de aprendizaje
<hr>

- Introduction of Model-Building Strategy of a time series
- practice plot skill in python

## 1. Introduction
<hr>

## 2. Introduction
<hr>

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -i df -w 5 -h 5 --units in -r 200